In [ ]:
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def tokenize_words(text):
  text = text.lower()

  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(text)

  filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
  return " ".join(filtered)

In [ ]:
#file = open("/content/drive/MyDrive/audio_dataset/tr/data.txt").read()

tokens =  open("/content/drive/MyDrive/audio_dataset/tr/tokens.txt").read()#tokenize_words(file)

characters = sorted(list(set(tokens)))



In [ ]:
char_to_num = dict((c, i) for i, c in enumerate(characters))
num_to_char = dict((i, c) for i, c in enumerate(characters))

In [ ]:
input_len = len(tokens)
vocab_len = len(characters)

seq_len = 100

x_data = []
y_data = []

In [ ]:
for i in range(0, (input_len) - seq_len, 1):
    in_seq = tokens[i:i + seq_len]

    out_seq = tokens[i + seq_len]

    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [ ]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 4044291


In [ ]:
X = numpy.reshape(x_data, (n_patterns, seq_len, 1))
X = X/float(vocab_len)

In [ ]:
numpy.save("codes.npy", X)

In [ ]:
y = np_utils.to_categorical(y_data)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [ ]:
model.fit(X, y, epochs=30, batch_size=256, callbacks=desired_callbacks)

Epoch 1/30
15799/15799 [==============================] - 1149s 71ms/step - loss: 2.3983

Epoch 00001: loss improved from inf to 2.11632, saving model to model_weights_saved.hdf5
Epoch 2/30
15799/15799 [==============================] - 1121s 71ms/step - loss: 1.7998

Epoch 00002: loss improved from 2.11632 to 1.76664, saving model to model_weights_saved.hdf5
Epoch 3/30
15799/15799 [==============================] - 1119s 71ms/step - loss: 1.6912

Epoch 00003: loss improved from 1.76664 to 1.67429, saving model to model_weights_saved.hdf5
Epoch 4/30
15799/15799 [==============================] - 1120s 71ms/step - loss: 1.6334

Epoch 00004: loss improved from 1.67429 to 1.62388, saving model to model_weights_saved.hdf5
Epoch 5/30
15799/15799 [==============================] - 1122s 71ms/step - loss: 1.5964

Epoch 00005: loss improved from 1.62388 to 1.59043, saving model to model_weights_saved.hdf5
Epoch 6/30
15799/15799 [==============================] - 1121s 71ms/step - loss: 1.5716


In [ ]:
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" ferent guys avalon sings song song heart cute female backing vocals violins piano feature maybe pian "
